<a href="https://colab.research.google.com/github/hhnafis/langchain_rag_project/blob/main/langchain_rag_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.10
    Uninstalling aiohttp-3.11.10:
      Successfully uninstalled aiohttp-3.11.10


**Setup Environment Variables**

In [2]:
from pinecone import Pinecone, ServerlessSpec
from google.colab import  userdata
pinecone_api_key = userdata.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

**Initializing Pinecone (*creating index*)**

In [3]:
import time

index_name = "langchain-rag-project"

pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

index = pc.Index(index_name)

**Use LangChain for RAG Workflow**

Use Google Gemini embeddings to vectorize documents

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
os.environ['GOOGLE_API_KEY'] = userdata.get("GOOGLE_API_KEY")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

**Setup the document loader**

In [6]:
# Installing the necessary packages
!pip install langchain-community
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Setup the document loader
file_path = '/content/sample_data/AI.txt'
loader = TextLoader(file_path) #Change file path if needed
documents = loader.load()

# Setup the splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)


**Embed and Store Documents in Pinecone**


In [8]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index,embedding=embeddings)

from uuid import uuid4
from langchain_core.documents import Document

# Iterating through the split documents and creating individual Document objects
for i, doc in enumerate(docs):
    document = Document(page_content=doc.page_content, metadata={"source": "AI.txt"})
    uuid = str(uuid4())  # Generated a unique ID for each document
    vector_store.add_documents(documents=[document], ids=[uuid]) # Added document to Pinecone

**Set Up Retriever**

In [10]:
results = vector_store.similarity_search(
    "Artifial Intelligence is revolutionizing our life",
    k=1,
    filter={"source": "AI.txt"},
)
for res in results:
    print(f"{res.page_content} [{res.metadata}]")

Artificial Intelligence (AI) has become a cornerstone of modern technology, revolutionizing industries and reshaping the way we live, work, and think. But AI's journey is as fascinating as its current applications. From early philosophical musings to cutting-edge neural networks, AI has undergone a remarkable evolution. This article delves into the milestones, breakthroughs, and concepts that have defined AI’s trajectory, offering insights into its past, present, and future. [{'source': 'AI.txt'}]


**Set Up Google Gemini Flash Model**


In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    verbose=True,
)

**Combine Retriever and LLM**

In [12]:
def answer_to_user(query: str):
    # Vector Search
    vector_results = vector_store.similarity_search(query, k=1)


    # Pass the model vector search + user query
    final_answer = llm.invoke(
        f"ANSWER THIS USER QUERY: {query}, Here are some references {vector_results}"
    )


    return final_answer

**Query the RAG System**

In [13]:
answer = answer_to_user("What is the history of artificial intelligence?")
answer.content

'The history of artificial intelligence (AI) has roots stretching back to the 17th century.  Mathematicians like Leibniz envisioned a "universal language" to systematically represent human reasoning, a concept that foreshadowed the development of symbolic logic – a crucial precursor to AI algorithms.\n\nThe field of AI as we know it today truly began to emerge in the mid-20th century (1940s-1950s), driven by significant advancements (the specifics of which are not detailed in the provided text).\n'

**Deploy the RAG system as an API using FastAPI**

In [20]:
#Complete RAG system deployment using FastAPI
!pip install fastapi uvicorn
#importing necessary libraries
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata
import time
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from uuid import uuid4
from langchain_core.documents import Document
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize FastAPI app
app = FastAPI(title="RAG API", description="Retrieval Augmented Generation API")


pinecone_api_key = userdata.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)
index_name = "langchain-rag-project"

os.environ['GOOGLE_API_KEY'] = userdata.get("GOOGLE_API_KEY")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_store = PineconeVectorStore(index=index,embedding=embeddings)

file_path = '/content/sample_data/AI.txt'
loader = TextLoader(file_path)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)


for i, doc in enumerate(docs):
    document = Document(page_content=doc.page_content, metadata={"source": "AI.txt"})
    uuid = str(uuid4())
    vector_store.add_documents(documents=[document], ids=[uuid])

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

def answer_to_user(query: str):
    vector_results = vector_store.similarity_search(query, k=1)

    context_content = vector_results[0].page_content if vector_results else ""
    prompt = f"""
    Question:
    {query}

    Context:
    {context_content}


    Response (Generated by Gemini):

    """
    final_answer = llm.invoke(prompt)

    return {"question":query, "answer":final_answer.content}


class Query(BaseModel):
    query: str
#API end point
@app.post("/query")
async def query_rag(query_data: Query):
    try:
        answer = answer_to_user(query_data.query)
        response = {
            "Question": query_data.query,
            "Response (Generated by Gemini)": answer.content,
        }
        return response
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing query: {str(e)}")

In [21]:
test_query = "What were the major milestones in AI development during the 20th century?"
test_answer = answer_to_user(test_query)

# Print the question and answer in the desired format
print(f"Question:\n{test_answer['question']}\n\nResponse (Generated by Gemini):\n\"{test_answer['answer']}\"")

Question:
What were the major milestones in AI development during the 20th century?

Response (Generated by Gemini):
"The 20th century witnessed several major milestones in AI development, although the field's progress was punctuated by periods of intense excitement followed by "AI winters" of reduced funding and interest.  Here are some key milestones:

* **1940s-1950s: The Birth of AI:**  The conceptual foundations of AI were laid.  Warren McCulloch and Walter Pitts's work on neural networks (1943) provided a theoretical model for computation in the brain. Alan Turing's work on computation and his famous Turing Test (1950) proposed a way to evaluate machine intelligence.  The Dartmouth Workshop in 1956, widely considered the "birth of AI," brought together leading researchers and formally established the field.  Early programs like the Logic Theorist (Allen Newell and Herbert Simon, 1955) and the General Problem Solver (Newell and Simon, 1957) demonstrated the potential of symbolic r